In [ ]:
import os
import numpy as np
import tifffile
from natsort import natsorted
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
folder_path = "E:/Temperature_images/RAW_2023-07-13/"  # Specify the path to your folder here
excel_file_path = "E:/Temperature_images/RAW_2023-07-13/Temperatures.xlsx"  # Specify the path to your Excel file here
section_to_delete = "air_satTemp"  # Specify the section you want to delete here

# Read the Excel file and extract the temperature column
df = pd.read_excel(excel_file_path)
temperature_column = df['Temperature'].tolist()

# Get the list of files in the folder and sort them naturally
files = natsorted(os.listdir(folder_path))

# Iterate over the files and rename them
for i, file_name in enumerate(files):
    if file_name.endswith('.tiff'):  # Check if the file has the desired extension
        # Extract the file name and extension
        file_base, file_extension = os.path.splitext(file_name)
        
        # Remove the section to delete from the file name
        new_file_name = file_base.replace(section_to_delete, "")

        # Construct the new file name with the temperature value before the extension
        new_file_name = f"{new_file_name[:15]}{str(temperature_column[i]).replace('.', ',')}{file_extension}"

        # Handle the case when the destination file already exists
        counter = 1
        while os.path.exists(os.path.join(folder_path, new_file_name)):
            # Add a suffix number to the new file name
            new_file_name = f"{file_base}_{counter}{file_extension}"
            counter += 1
        
        # Construct the new file path
        new_file_path = os.path.join(folder_path, new_file_name)
        
        # Rename the file
        os.rename(os.path.join(folder_path, file_name), new_file_path)



In [ ]:
# Set the path to the input directory
input_dir = "E:/Temperature_images/RAW_2023-07-13/"


# Get a list of all image files in the input directory
input_files = [f for f in os.listdir(input_dir) if f.endswith('.tiff')]
input_files = natsorted(input_files)

# Calculate the number of rows and columns for the grid
num_images = len(input_files)
num_rows = int(num_images**0.5)
num_cols = (num_images + num_rows - 1) // num_rows

# Create the grid of subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8))

# Iterate over the image files and display them in the grid
for i, filename in enumerate(input_files):
    # Load the image
    image = tifffile.imread(os.path.join(input_dir, filename))

    # Determine the subplot indices
    row_idx = i // num_cols
    col_idx = i % num_cols

    # Display the image in the corresponding subplot
    axes[row_idx, col_idx].imshow(image, vmin=256,vmax=4065)
    axes[row_idx, col_idx].set_title(filename[15:])
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5, wspace=0.3)

# Display the grid of images
plt.show()



In [ ]:
# Define the threshold value
threshold_value = 256

# Create a list to store the thresholded images
thresholded_images = []

# Calculate the number of rows and columns for the grid
num_images = len(input_files)
num_rows = int(num_images ** 0.5)
num_cols = (num_images + num_rows - 1) // num_rows

# Create the grid of subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8))

# Iterate over the image files and display them in the grid
for i, filename in enumerate(input_files):
    # Load the image
    image = tifffile.imread(os.path.join(input_dir, filename))

    # Apply the threshold
    thresholded_image = np.where(image > threshold_value, image, 0)
    thresholded_image_nan = np.where(thresholded_image == 0, np.nan, thresholded_image)

    thresholded_images.append(thresholded_image_nan)

    # Determine the subplot indices
    row_idx = i // num_cols
    col_idx = i % num_cols

    # Display the thresholded image in the corresponding subplot
    axes[row_idx, col_idx].imshow(thresholded_image, cmap='gray', vmin=256,vmax=4065)
    axes[row_idx, col_idx].set_title(filename[15:])
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Display the grid of images
plt.show()


In [ ]:
# Set output directory
output_dir = "E:/Temperature_images/RAW_2023-07-13/Threshold_images/"

# Save Thresholded images
for i, filename in enumerate(input_files):
    output_path = os.path.join(output_dir, filename)
    tifffile.imwrite(output_path, thresholded_images[i])

In [ ]:
# Set the path to the directory containing the thresholded images
thresholded_dir = 'E:/Temperature_images/RAW_2023-07-13/Threshold_images/'

# Get a list of all thresholded image files in the directory
thresholded_files = [f for f in os.listdir(thresholded_dir) if f.endswith('.tiff')]
thresholded_files = natsorted(thresholded_files)

test_thresholded_image = tifffile.imread(os.path.join(thresholded_dir,thresholded_files[0]))
crop_test_thresholded_image = test_thresholded_image[1500:3000,0:1800] 
plt.imshow(crop_test_thresholded_image, cmap="inferno")
print(np.shape(crop_test_thresholded_image))


In [ ]:
# Set the input directory path
input_dir = "E:/Temperature_images/RAW_2023-07-13/"
output_path = "E:/Temperature_images/RAW_2023-07-13/Stacks/"

# Create the output directory if it doesn't already exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Get a list of all TIFF files in the input directory
input_files = [f for f in os.listdir(input_dir) if f.endswith('.tiff')]
input_files = natsorted(input_files)
# Reshape array of input files
files_list = np.reshape(input_files, (-1, 1))
print(files_list)
# Initialize empty lists for the red and green stacks
red_stack = []
green_stack = []
blue_stack = []
raw_stack =[]

# Loop through each image file in the input directory
for file in input_files:
    # Load the image using tifffile
    img = tifffile.imread(os.path.join(input_dir, file))

    ### Set Crop dimension (OPTIONAL) ###
    img = img[1500:3000,0:1800] 
    
    # Extract the red and green channels
    red_channel = img[1::2, 1::2]
    blue_channel = img[0::2, 0::2]
    green_channel_1 = img[0::2, 1::2]
    green_channel_2 = img[1::2, 0::2]
    green_channel = np.add(green_channel_1, green_channel_2) // 2
     # Add the red and green channels to the corresponding stacks
    blue_stack.append(blue_channel)
    red_stack.append(red_channel)
    green_stack.append(green_channel)
    raw_stack.append(img)

# Stack the red and green channel lists along the third axis to create the corresponding stacks
red_stack = np.stack(red_stack, axis=0)
green_stack = np.stack(green_stack, axis=0)
blue_stack =np.stack(blue_stack, axis=0)

# Save the red and green stacks to separate TIFF files
tifffile.imwrite(os.path.join(output_path, "stacked_red_channel.tiff"), red_stack)
tifffile.imwrite(os.path.join(output_path, "stacked_green_channel.tiff"), green_stack)
tifffile.imwrite(os.path.join(output_path, "stacked_blue_channel.tiff"), blue_stack)




In [ ]:
# Set the path to the directory containing the image stacks
stack_dir = 'E:/Temperature_images/RAW_2023-07-13/Stacks/'

# Get a list of all TIFF files in the input directory
files = [f for f in os.listdir(stack_dir) if f.endswith('.tiff')]
print(files)

green_stack = tifffile.imread(os.path.join(stack_dir, files[1]))
red_stack = tifffile.imread(os.path.join(stack_dir, files[2]))
blue_stack = tifffile.imread(os.path.join(stack_dir, files[0]))

#Get length of image stacks
num_images_green = len(green_stack)
num_images_red = len(red_stack)
num_images_blue =len(blue_stack)
if num_images_green == num_images_red and num_images_blue:
    print("Equal stacks")
    num_images = num_images_green
else:
    print("unequal stacks")

print("stack=",num_images)

In [ ]:
#FOR TEMPERATURE CALIBRATION: Loop through all red/blue pairs of images in the stacks an get the RED/BLUE intensity ratio
mean_red_intensity =[]
mean_green_intensity = []
mean_blue_intensity = []
std_red_intensity = []
std_green_intensity = []
std_blue_intensity =[]
red_blue_ratio = []
mean_red_blue_ratio =[]
red_blue_std = []

for i in range(num_images):

    mean_red_intensity.append(np.mean(red_stack[i]))
    mean_green_intensity.append(np.mean(green_stack[i]))
    mean_blue_intensity.append(np.mean(blue_stack[i]))
    std_red_intensity.append(np.std(red_stack[i]))
    std_green_intensity.append(np.std(green_stack[i]))
    std_blue_intensity.append(np.std(blue_stack[i]))

    # Calculate the intensity ratio between the two images
    ratio = red_stack[i] / blue_stack[i]
    mean_red_blue_ratio.append(np.mean(ratio))
    red_blue_ratio.append(ratio)
    red_blue_std.append(np.std(ratio))


In [ ]:
# Define number of sections for each image to calculate mean ratios
num_sections = 3

# Calculate the height of each section
section_height = red_stack.shape[1] // num_sections

# Loop over eack image in the ratio stack and divide it into sections
ratio_section_means = []
ratio_sections = []

# CHANGE DEPENDING ON CALIBRATION
for image in red_blue_ratio:
    sections = [image[i*section_height:(i+1)*section_height, :] for i in range(num_sections)]
    ratio_sections.append(sections)

    # Calculate the mean ratio of eacjh section and append it to the list
    ratio_section_mean = [np.nanmean(section) for section in sections]
    ratio_section_means.append(ratio_section_mean)

# Reshape array of mean ratios to 
ratio_section_means = np.reshape(ratio_section_means, (-1, num_sections))
print(ratio_section_means)

In [ ]:
# Define the image index to show
image_idx = 0

# Get an example image from the green stack and its sections to see how its divided
image = red_stack[image_idx]
sections = ratio_sections[image_idx]

for section_idx in range(num_sections):
    section = sections[section_idx]
    plt.imshow(section, cmap="gray")
    plt.title(f'Image {image_idx}, Section {section_idx}')
    plt.show()


In [ ]:
# Calculate the mean of means for each image and the standard deviation
ratio_mean_of_means = []
ratio_std_of_means =[]

for i in range(len(ratio_section_means)):
    mean_ratio = np.nanmean(ratio_section_means[i])
    std_ratio = np.nanstd(ratio_section_means[i])
    ratio_mean_of_means.append(mean_ratio)
    ratio_std_of_means.append(std_ratio)

print(ratio_mean_of_means)
print(ratio_std_of_means)

In [ ]:
# Set the path to the directory containing the image filenames
files_dir = 'E:/Temperature_images/RAW_2023-07-13/'
files = [f for f in os.listdir(files_dir) if f.endswith('.tiff')]

# Sort filenames to match with stack index
files = natsorted(files)

# Initialize an empty dataframe
df_ratios = pd.DataFrame(columns=['Temperature/O2', 'Intensity Ratio', 'Intensity Red', 'Intensity Green', 'Intensity Blue', 'Std Ratio','Std Red','Std Green', 'Std Blue'])

# Add filenames and corresponding red/green ratio to the dataframe
for i in range(num_images):
    ratio_df = ratio_mean_of_means[i]
    std_df = ratio_std_of_means[i]
    blue_intensity = mean_blue_intensity[i]
    green_intensity = mean_green_intensity[i]
    red_intensity = mean_red_intensity[i]
    blue_std = std_blue_intensity[i]
    green_std = std_green_intensity[i]
    red_std = std_red_intensity[i]
    filename = os.path.splitext(files[i])[0][15:]
    numeric_num = float(filename.replace(",", ".")) 
    df_ratios.loc[i] = [numeric_num, ratio_df, red_intensity, green_intensity, blue_intensity, std_df, red_std, green_std, blue_std]

#df_ratios['Intensity Ratio'] = df_ratios['Intensity Ratio'].round(4)

# Print the dataframe
print(df_ratios)

In [ ]:
#Save the DataFrame as a CSV file
df_ratios.to_csv('E:/Temperature_images/RAW_2023-07-13/red_blue_ratio.csv',header=True, index=False)

In [ ]:
df_ratios = pd.read_csv('E:/Temperature_images/RAW_2023-07-13/red_blue_ratio.csv')
print(df_ratios)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score

# Assuming df_ratios_adj is your DataFrame and it's already defined

# Create the figure and axes
fig, ax = plt.subplots(figsize=(12, 8))

# Calculate linear regression coefficients for the intensity ratio
coef_ratio = np.polyfit(df_ratios['Temperature/O2'], df_ratios['Intensity Red'], 1)

# Create linear regression line for the ratio
line_ratio = np.polyval(coef_ratio, df_ratios['Temperature/O2'])

# Linear regression equation as a string
eq_ratio = f'y = {coef_ratio[0]:.2f}x + {coef_ratio[1]:.2f}'

# Calculate R^2 value
r_squared = r2_score(df_ratios['Intensity Red'], line_ratio)
eq_full = f'{eq_ratio}\n$R^2 = {r_squared:.3f}$'

# Plotting the ratio data
ax.scatter(df_ratios['Temperature/O2'], df_ratios['Intensity Red'], label='Red/Blue Red', marker="o", color="navy" ,linewidths=2, edgecolors="black", s=150)
ax.errorbar(df_ratios['Temperature/O2'], df_ratios['Intensity Red'], yerr=df_ratios['Std Red'], fmt='none', capsize=4, ecolor='black')
ax.plot(df_ratios['Temperature/O2'], line_ratio, color='r', linewidth=5)
ax.annotate(eq_full, xy=(0.50, 0.80), xycoords='axes fraction', color="black", fontsize=30)

# Set labels and formatting
ax.set_ylabel('Red Channel Intensity', fontsize=35)
ax.set_xlabel('Temperature (°C)', fontsize=35)
ax.tick_params(axis='y', labelsize=25)
ax.tick_params(axis='x', labelsize=25)
#ax.set_yticks(np.arange(5, 10, 1))
ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray', alpha=0.7)

# Legend
#ax.legend(loc='upper right', fontsize=125)

# Adjust spacing and show plot
fig.tight_layout()
plt.savefig('E:/Temperature_images/RAW_2023-07-13/Temperature_cali.png', dpi=300)
plt.show()
